In [1]:
import sys
import os
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER')
import fastText
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import pickle, threading

from keras.utils import to_categorical
import linecache
from keras.callbacks import Callback

epochs = 50
trainable=True

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [43]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [44]:
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')
print(caseEmbeddings.shape)

(8, 8)


In [45]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [46]:
characters={}

with open ('/home/jindal/notebooks/jindal/NER/language_model/toy_dataset_sequences_shuffled') as f:
    for line in f:
        for word in line:
            word =str(word)
            for char in word:
#                 print(char)
                characters[char]=True

In [47]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [48]:
toy_vocab ={}

with open('toy_dataset_sequences') as f:
    for line in f:
#         print(line)
        line = line.split()
#         print(len(line))
        for word in line:
#             word = word
#             print(word)
            toy_vocab[word]=True

In [49]:
word2Idx={}
for word in toy_vocab.keys():
    word2Idx[word] = len(word2Idx)

In [50]:
vocab_size = len(word2Idx.keys())+1
print(vocab_size)

4851


In [51]:
words_input = Input(shape=(None,300), dtype='float32',name='words_input')
# words = Embedding(input_dim =50, output_dim=300, trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
# model.summary()

In [52]:
model.load_weights('german_lm.h5')

In [12]:
for layer in model.layers:
    print(layer.name)

char_input
char_embedding
dropout1
time_distributed_1
maxpool
time_distributed_2
words_input
dropout_1
concatenate_1
bidirectional_1
dense_1


In [53]:
model.layers.pop()

In [54]:
for layer in model.layers:
    print(layer.name)

char_input
char_embedding
dropout1
time_distributed_5
maxpool
time_distributed_6
words_input
dropout_3
concatenate_3
bidirectional_3


In [97]:
characters={}
labels = {}
with open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training.txt') as f:
# with open('/home/gwiedemann/notebooks/OffLang/sample_train.txt') as f:
    for line in f:
        text, label_simple, label_complex = line.split('\t')
        labels[label_simple]=True
        for word in text:
            for char in word:
                characters[char]=True
                

In [98]:
label2Idx = {}
for label in labels:
    label2Idx[label] = len(label2Idx)

In [99]:
print(label2Idx)

{'OFFENSE': 0, 'OTHER': 1}


In [100]:
output = Dense(len(label2Idx), activation = 'softmax')(model.layers[-1].output)

In [101]:
final_model = Model(inputs=model.input, outputs=[output])

In [63]:
for layer in final_model.layers:
    print(layer.name)

char_input
char_embedding
dropout1
time_distributed_5
maxpool
time_distributed_6
words_input
dropout_3
concatenate_3
bidirectional_3
dense_5


In [111]:
final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3300        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [24]:
out_file1 = open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt','wb')
out_file2 = open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_complex_labels.txt','wb')
with open('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training.txt', ) as f:
    for line in f:
        line = line.split('\t')
#         print(len(line))
        text, label_simple, label_complex = line
        simple = [text, label_simple]
        compl = [text, label_complex]
        string_simple = '\t'.join(simple)+' \n'
        string_complex = '\t'.join(compl)+' \n'
        out_file1.write(string_simple.encode())
        out_file2.write(string_complex.encode())
out_file1.close()
out_file2.close()

In [65]:
line_number=0

In [26]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [66]:
print(labels)

{'OFFENSE': True, 'OTHER': True}


In [67]:
print(label2Idx)

{'OFFENSE': 0, 'OTHER': 1}


In [107]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [69]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [31]:
linecache.getline('/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt', 64)

'@Miquwarchar Hat die jemand dafür angezeigt?\tOTHER \n'

In [103]:
def my_generator(file :"input training file", batch_size):
    
    global line_number, lock

    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []
        total_lines = file_len(file)
        for i in range(batch_size):
            
#             lock.acquire()
            
            index = line_number%total_lines +1
            line_number+=1
#             lock.release()
#             index = random.choice(len(features), 1)[0]
            line = linecache.getline(file, index)
            if len(line.split('\t'))!=3:
                continue
            text, label_simple, label_complex = line.split('\t')
#             assert index<=100
#             print(text)
            temp_casing = []
            temp_char=[]
            temp_word=[]
#             print(line)
#             print(len)
            for word in text.split():
#                 print(word)
                casing =getCasing(word, case2Idx)
#                 print(casing)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
#                 print(temp_char2)
#                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
#                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
#                 print(len(word_vector))
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
#             print(temp_word)
#             print(len(temp_word))
#             print(len(temp_casing))
#             print(temp_char)
#             print(len(temp_char))
#             print(label2Idx[label])
#             print(index)
#             print(" **************** ")
            word_embeddings.append(temp_word)
#             print((len(word_embeddings), len(word_embeddings[0]), len(word_embeddings[0][0])))

            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(label2Idx[label_simple])
#             print(np.array(word_embeddings))
#             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
#             batch_labels[i] = labels[index]
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], to_categorical(output_labels, len(label2Idx)))

In [77]:
line_number=0
for inp, output in my_generator('/home/gwiedemann/notebooks/OffLang/sample_train.txt',32):
    continue
#     print(inp[0].shape)
#     print(inp[1].shape)
#     print(inp[2].shape)
#     print(output.shape)

KeyboardInterrupt: 

In [104]:
def validation_data_generator(file, batch_size):
    
    global line_number_validation, lock

    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []
        total_lines = file_len(file)
        for i in range(batch_size):
            
#             lock.acquire()
            
            index = line_number_validation%total_lines +1
            line_number_validation+=1
#             lock.release()
#             index = random.choice(len(features), 1)[0]
            line = linecache.getline(file, index)
            if len(line.split('\t'))!=3:
                continue
            text, simple_label, complex_label = line.split('\t')
#             assert index<=100
#             print(text)
            temp_casing = []
            temp_char=[]
            temp_word=[]
#             print(line)
#             print(len)
            for word in text.split():
#                 print(word)
                casing =getCasing(word, case2Idx)
#                 print(casing)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    temp_char2.append(char2Idx[char])
                temp_char2 = np.array(temp_char2)
#                 print(temp_char2)
#                 temp_char2 = pad_sequences(temp_char2, 52, padding='post')
#                 temp_char.append(pad_sequences(temp_char2, 52, padding='post'))
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
#                 print(len(word_vector))
                temp_word.append(word_vector)
            temp_char = pad_sequences(temp_char, 52)
#             print(temp_word)
#             print(len(temp_word))
#             print(len(temp_casing))
#             print(temp_char)
#             print(len(temp_char))
#             print(label2Idx[label])
#             print(index)
#             print(" **************** ")
            word_embeddings.append(temp_word)
#             print((len(word_embeddings), len(word_embeddings[0]), len(word_embeddings[0][0])))

            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(label2Idx[label_simple])
#             print(np.array(word_embeddings))
#             batch_features[i] = process_features(line, window_size-1, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
#             batch_labels[i] = labels[index]
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], to_categorical(output_labels, len(label2Idx)))
            

In [142]:
print(to_categorical(1,3))


[0. 1. 0.]


In [ ]:
line_number=0
line_number_validation=0
# training_file_name = '/home/jindal/notebooks/jindal/NER/language_model/germeval2018.training_simple_labels.txt'
training_file_name = '/home/gwiedemann/notebooks/OffLang/sample_train.txt'
dev_file_name = '/home/gwiedemann/notebooks/OffLang/sample_dev.txt'
validation_file_name = '/'
final_model.fit_generator(my_generator(training_file_name, 1)
                         , epochs =4 , steps_per_epoch = file_len(training_file_name)//1, validation_data=validation_data_generator(dev_file_name, 1),
                         validation_steps=807)

In [125]:
idx2Label={}
for key, value in label2Idx.items():
    idx2Label[value]=key

In [144]:
with open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt') as f:
    for line in f:
#         print(line.split('\t')[0], end=' ')
        word_embeddings=[]
        case_embeddings=[]
        char_embeddings=[]
        if len(line.split('\t'))!=3:
                continue
        text, simple_label, complex_label = line.split('\t')
        temp_casing = []
        temp_char=[]
        temp_word=[]
        for word in text.split():
            casing =getCasing(word, case2Idx)
            temp_casing.append(casing)
            temp_char2=[]
            for char in word:
                temp_char2.append(char2Idx[char])
            temp_char2 = np.array(temp_char2)
            temp_char.append(temp_char2)
            word_vector = ft.get_word_vector(word.lower())
            temp_word.append(word_vector)
        temp_char = pad_sequences(temp_char, 52)
        word_embeddings.append(temp_word)
        case_embeddings.append(temp_casing)
        char_embeddings.append(temp_char)
#         output_labels.append(label2Idx[label_simple])
        inp= [np.array([temp_word]), np.array([temp_casing]), np.array([temp_char])]
#         yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], to_categorical(output_labels, len(label2Idx)))
#         print(final_model.predict_classes(inp))
        print(final_mod.predict(inp, batch_size=1)[0].argmax(-1))
#         y_classes = y_prob.argmax(axis=-1)
        


3496
263
3496
963
2261
2261
963
2261
2261
3536
3496
3105
963
2261
3536
3893
2261
2261
963
963
3496
963
263
3536
2261
2057
2057
963
3496
963
2261
3496
3496
2261
3536
3496
2057
3496
963
3536
963
3496
2261
963
2057
2261
2261
2057
963
2261
3496
3536
3496
2261
2261
781
3496
2261
3496
2261
2261
2261
2933
3496
963
3536
2261
3496
3496
3536
611
963
963
2261
3496
963
3496
263
3496
3536
3221
3536
3105
781
2261
2261
2057
3496
963
963
2762
2261
3304
963
2261
2261
3536
3496
3496
2261
2261
963
2261
3536
2261


KeyboardInterrupt: 